In [1]:
import pandas as pd
from hnmchallenge.data_reader import DataReader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from hnmchallenge.dataset import Dataset
from hnmchallenge.filtered_dataset import FilterdDataset
from hnmchallenge.stratified_dataset import StratifiedDataset
from hnmchallenge.constant import *
import datetime

In [2]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG)

In [10]:
dataset = StratifiedDataset()
dr = DataReader()

In [11]:
# creating H&M features in holdin 
feature = dataset.get_holdin()

In [12]:
item_feature=feature.drop(["customer_id","sales_channel_id","price"], axis=1)

In [13]:
item_feature

,t_dat,article_id
0,2018-09-20,0
1,2018-09-20,0
2,2019-08-05,0
3,2019-08-16,644
4,2019-10-06,1952
...,...,...
10955395,2020-09-22,17478
10955396,2020-09-22,10033
10955397,2020-09-22,7580
10955398,2020-09-22,1765


In [16]:
item_feature1=item_feature[item_feature["t_dat"]>="2020-09-01"]

In [19]:
item_feature1.reset_index(inplace=False)

In [23]:
item_feature1=item_feature1.drop(["index"],axis=1)

In [24]:
item_feature1

,t_dat,article_id
0,2020-09-14,4705
1,2020-09-01,2873
2,2020-09-01,16951
3,2020-09-01,6749
4,2020-09-01,14886
...,...,...
174687,2020-09-22,17478
174688,2020-09-22,10033
174689,2020-09-22,7580
174690,2020-09-22,1765


In [26]:
duplicated_rows = item_feature[item_feature.duplicated(subset=[ DEFAULT_ITEM_COL])]
count_mb = duplicated_rows.groupby(DEFAULT_ITEM_COL).count()
count_mb = count_mb.reset_index()[[DEFAULT_ITEM_COL, "t_dat"]].rename(
    columns={"t_dat": "count"}
)

In [27]:
count_mb

,article_id,count
0,0,3300
1,1,2085
2,2,1082
3,3,1875
4,4,5343
...,...,...
20389,21987,1
20390,21993,3
20391,21997,2
20392,22001,3


In [34]:
count_mb["article_id"].nunique()

20394

In [29]:
duplicated_rows = item_feature1[item_feature1.duplicated(subset=[ DEFAULT_ITEM_COL])]
count_mb1 = duplicated_rows.groupby(DEFAULT_ITEM_COL).count()
count_mb1 = count_mb1.reset_index()[[DEFAULT_ITEM_COL, "t_dat"]].rename(
    columns={"t_dat": "count_1st_sept"}
)

In [30]:
count_mb1

,article_id,count_1st_sept
0,4,8
1,5,40
2,6,7
3,7,277
4,8,135
...,...,...
11371,21987,1
11372,21993,3
11373,21997,2
11374,22001,3


In [35]:
item_features=count_mb.join(count_mb1.set_index('article_id'), on='article_id')

In [36]:
item_features['count_1st_sept']=item_features['count_1st_sept'].fillna(0)

In [37]:
item_features

,article_id,count,count_1st_sept
0,0,3300,0.0
1,1,2085,0.0
2,2,1082,0.0
3,3,1875,0.0
4,4,5343,8.0
...,...,...,...
20389,21987,1,1.0
20390,21993,3,3.0
20391,21997,2,2.0
20392,22001,3,3.0


In [ ]:
dr = DataReader()
feature.to_feather(
        dr.get_preprocessed_data_path() / "stratified_holdin_item_dataset.feather"
    )